# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_path = "../WeatherPy/cities_data_file.csv"
cities_weather_df = pd.read_csv(file_path)
cities_weather_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,0,rikitea,19.0,PF,1.620629e+09,77.0,-23.1203,-134.9692,77.74,15.12
1,1,ancud,3.0,CL,1.620629e+09,93.0,-41.8697,-73.8203,41.00,8.05
2,2,kapaa,20.0,US,1.620629e+09,73.0,22.0752,-159.3190,75.20,13.80
3,3,cabo san lucas,0.0,MX,1.620629e+09,63.0,22.8909,-109.9124,79.00,13.56
4,4,bluff,0.0,NZ,1.620629e+09,77.0,-46.6000,168.3333,59.00,5.01
...,...,...,...,...,...,...,...,...,...,...
577,577,batagay-alyta,75.0,TO,1.620630e+09,78.0,-18.6500,-173.9833,75.20,11.50
578,578,coswig,100.0,VN,1.620630e+09,57.0,21.3167,103.9000,84.63,2.06
579,579,tunduru,100.0,RU,1.620630e+09,88.0,66.0000,111.5000,28.78,13.85
580,580,rock sound,99.0,MV,1.620630e+09,70.0,2.6708,72.8944,85.26,6.42


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(g_key)

In [4]:
locations = cities_weather_df[["Latitude", "Longitude"]]
humidity = cities_weather_df["Humidity"].astype(float)
max_humidity = max(humidity)

In [5]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_humidity, 
                                 point_radius=3.5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Perfect temperature (Between 70 and 80 Degrees)
perfect_conditions = cities_weather_df[(cities_weather_df["Max Temp"] >70) 
                                       & (cities_weather_df["Max Temp"] < 80) 
                                       & (cities_weather_df["Wind Speed"] < 10) 
                                       & (cities_weather_df["Cloudiness"] == 0)]

perfect_conditions

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
14,14,ajdabiya,0.0,LY,1.620629e+09,45.0,30.7554,20.2263,75.92,7.20
99,99,upernavik,0.0,AU,1.620629e+09,46.0,-35.3500,150.4667,72.00,6.91
158,158,khonuu,0.0,AO,1.620629e+09,73.0,-15.1961,12.1522,76.14,2.24
171,171,bodmin,0.0,SN,1.620629e+09,64.0,16.3333,-15.0000,71.60,2.30
299,299,geraldton,0.0,AU,1.620629e+09,49.0,-32.7167,152.1500,73.40,9.22
378,378,nayoro,0.0,LY,1.620629e+09,45.0,29.0331,21.5482,75.18,3.15
449,449,zhicheng,0.0,SV,1.620630e+09,88.0,13.2833,-88.5500,75.20,4.61
455,455,padang,0.0,ZW,1.620630e+09,46.0,-16.7725,31.5838,70.48,5.44
502,502,ca mau,0.0,RU,1.620630e+09,40.0,57.1951,56.9091,71.06,8.37
516,516,santa catalina,0.0,BR,1.620630e+09,100.0,-5.5183,-47.4611,75.20,2.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = pd.DataFrame(perfect_conditions)
hotel_df["Hotel Name"] = ""
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
14,14,ajdabiya,0.0,LY,1.620629e+09,45.0,30.7554,20.2263,75.92,7.20,
99,99,upernavik,0.0,AU,1.620629e+09,46.0,-35.3500,150.4667,72.00,6.91,
158,158,khonuu,0.0,AO,1.620629e+09,73.0,-15.1961,12.1522,76.14,2.24,
171,171,bodmin,0.0,SN,1.620629e+09,64.0,16.3333,-15.0000,71.60,2.30,
299,299,geraldton,0.0,AU,1.620629e+09,49.0,-32.7167,152.1500,73.40,9.22,
378,378,nayoro,0.0,LY,1.620629e+09,45.0,29.0331,21.5482,75.18,3.15,
449,449,zhicheng,0.0,SV,1.620630e+09,88.0,13.2833,-88.5500,75.20,4.61,
455,455,padang,0.0,ZW,1.620630e+09,46.0,-16.7725,31.5838,70.48,5.44,
502,502,ca mau,0.0,RU,1.620630e+09,40.0,57.1951,56.9091,71.06,8.37,
516,516,santa catalina,0.0,BR,1.620630e+09,100.0,-5.5183,-47.4611,75.20,2.17,


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
